
# Clinic No-Show — Verification Notebook (Local **and** BigQuery)
Use this notebook to recompute the findings locally **or** reproduce them in **BigQuery**.

**You need** the 4 CSVs (or the all-in-one ZIP):
- `appointments_clean.csv`
- `weather.csv`
- `twitter_agg.csv`
- `zip_city.csv` (QA only)


In [ ]:

# Base paths (works in Colab or local Jupyter)
from pathlib import Path
import pandas as pd, numpy as np, os

DATA_DIR = Path("/content/data")  # Change to your folder if running locally
OUTPUT_DIR = DATA_DIR / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_rows", 50)
pd.set_option("display.float_format", lambda x: f"{x:0.1f}")
print("DATA_DIR:", DATA_DIR.resolve())


## Option A (Colab): Upload ZIP or individual CSVs

In [ ]:

# If running in Colab, uncomment and run this cell to upload ZIP or CSVs.
try:
    from google.colab import files
    uploaded = files.upload()
    import zipfile, io
    for name, content in uploaded.items():
        if name.endswith(".zip"):
            with zipfile.ZipFile(io.BytesIO(content)) as zf:
                zf.extractall(DATA_DIR)
        else:
            with open(DATA_DIR / name, "wb") as f:
                f.write(content)
    print("Uploaded to:", DATA_DIR)
except Exception:
    print("Upload helper not available (likely not in Colab). Use Option B (local path).")



## Option B (local): Point to your folder
If running locally, set `DATA_DIR` above to your CSV folder (e.g., `C:/Users/you/Downloads/clinic_project`) and re-run the next cells.


## Local path — recompute findings from CSVs

In [ ]:

required = ["appointments_clean.csv","weather.csv","twitter_agg.csv"]
missing = [fn for fn in required if not (DATA_DIR/fn).exists()]
if missing:
    raise FileNotFoundError(f"Missing files in {DATA_DIR}: {missing}")

appts = pd.read_csv(DATA_DIR/"appointments_clean.csv")
wx    = pd.read_csv(DATA_DIR/"weather.csv")
tw    = pd.read_csv(DATA_DIR/"twitter_agg.csv")

# Feature engineering & joins (mirror of BigQuery logic)
appts["dt"] = pd.to_datetime(appts["date"])
appts["booked_dt"] = pd.to_datetime(appts["booked_date"])
appts["city_up"] = appts["city"].str.upper()
appts["no_show"] = (appts["status"].str.lower()=="no_show").astype(int)
appts["hour"] = appts["time"].str.slice(0,2).astype(int)
appts["hour_bucket"] = appts["hour"].apply(lambda h: "AM" if 7<=h<=11 else ("PM" if 12<=h<=16 else "Late"))
appts["lead_time_days"] = (appts["dt"] - appts["booked_dt"]).dt.days.clip(lower=0)

wx["dt"] = pd.to_datetime(wx["dt"]); wx["city_up"]=wx["city"].str.upper()
tw["dt"] = pd.to_datetime(tw["dt"]); tw["city_up"]=tw["city"].str.upper()

fact = appts.merge(wx[["dt","city_up","tmax_c","tmin_c","prcp_mm"]], on=["dt","city_up"], how="left") \            .merge(tw[["dt","city_up","tweet_count_symptoms","sentiment_avg"]], on=["dt","city_up"], how="left")

fact["temp_bucket"] = fact["tmax_c"].apply(lambda t: "Hot" if t>=30 else ("Cold" if t<=5 else "Mild"))
pth = fact["tweet_count_symptoms"].quantile(0.75)
fact["chatter_bucket"] = np.where(fact["tweet_count_symptoms"]>=pth, "High", "Normal")

print("Overall no-show %:", round(fact["no_show"].mean()*100,1))


In [ ]:

# Lead time
lead = (fact.assign(lead_bucket=fact["lead_time_days"].apply(lambda x: "0-2" if x<=2 else ("3-7" if x<=7 else "8+")))
            .groupby("lead_bucket")["no_show"].mean().mul(100).round(1).reindex(["0-2","3-7","8+"]).reset_index())
display(lead); lead.to_csv(OUTPUT_DIR/"leadtime_rates_recomputed.csv", index=False)

# Weather × time
wx_time = fact.groupby(["temp_bucket","hour_bucket"])["no_show"].mean().mul(100).round(1).reset_index()
display(wx_time.sort_values(["temp_bucket","hour_bucket"])); wx_time.to_csv(OUTPUT_DIR/"weather_time_matrix_recomputed.csv", index=False)

# Chatter
chatter = fact.groupby("chatter_bucket")["no_show"].mean().mul(100).round(1).reindex(["Normal","High"]).reset_index()
display(chatter); chatter.to_csv(OUTPUT_DIR/"chatter_effect_recomputed.csv", index=False)

# Top segments
top = fact.groupby(["clinic","visit_type","hour_bucket","temp_bucket"])["no_show"].agg(['mean','count']).reset_index()
top["no_show_rate_pct"] = (top["mean"]*100).round(1)
top = top[top["count"]>=20].sort_values("no_show_rate_pct", ascending=False).head(15)
display(top[["clinic","visit_type","hour_bucket","temp_bucket","no_show_rate_pct","count"]])
top[["clinic","visit_type","hour_bucket","temp_bucket","no_show_rate_pct","count"]].to_csv(OUTPUT_DIR/"top_segments_recomputed.csv", index=False)

# Overall (write file)
overall = round(fact["no_show"].mean()*100,1)
with open(OUTPUT_DIR/"overall_no_show.txt","w") as f: f.write(str(overall))


### Verify against slide numbers (±0.5 ppt)

In [ ]:

expected = {
    "lead": {"0-2": 19.2, "3-7": 15.7, "8+": 17.2},
    "chatter": {"Normal": 16.6, "High": 18.6},
    "overall": 17.1,
    "wx_time_subset": {("Hot","AM"): 17.3, ("Hot","PM"): 18.7, ("Mild","AM"): 16.1, ("Mild","PM"): 17.3},
}
tol = 0.5
def close(a,b,t=tol): return abs(float(a)-float(b)) <= t

lead_check = {row["lead_bucket"]: float(row["no_show"]) for _, row in lead.iterrows()}
chat_check = {row["chatter_bucket"]: float(row["no_show"]) for _, row in chatter.iterrows()}
wx_dict = {(r["temp_bucket"], r["hour_bucket"]): float(r["no_show"]) for _,r in wx_time.iterrows()}

print("Lead-time PASS:", all(k in lead_check and close(lead_check[k], v) for k,v in expected["lead"].items()), lead_check)
print("Chatter   PASS:", all(k in chat_check and close(chat_check[k], v) for k,v in expected["chatter"].items()), chat_check)
print("Overall   PASS:", close(overall, expected["overall"]), overall)
print("Wx×Time   PASS:", all(k in wx_dict and close(wx_dict[k], v) for k,v in expected["wx_time_subset"].items()))



---

# BigQuery (cloud) path — end-to-end
This section loads the same CSVs into **BigQuery**, creates the standardized tables and modeling view, and runs the confirmation queries.

**What you need:**
- Google account with access to a GCP project
- Project ID (e.g., `my-gcp-project`)
- The four CSVs already in this notebook's `DATA_DIR` (use the upload cell above if needed).


In [ ]:

# Install libraries (Colab-friendly). If running locally, you may already have these.
!pip -q install google-cloud-bigquery pandas-gbq pyarrow >/dev/null

# Authenticate (Colab) and set the project
try:
    from google.colab import auth
    auth.authenticate_user()
    print("Authenticated.")
except Exception as e:
    print("Colab auth helper not available. If local, ensure application-default credentials are set.")

import os
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT") or input("Enter your GCP PROJECT_ID: ").strip()
print("Using PROJECT_ID:", PROJECT_ID)


In [ ]:

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

dataset_id = f"{PROJECT_ID}.clinic_ops"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
client.create_dataset(dataset, exists_ok=True)
print("Dataset ready:", dataset_id)


In [ ]:

from google.cloud import bigquery

def load_csv_to_bq(table_name, csv_path):
    table_id = f"{dataset_id}.{table_name}"
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        field_delimiter=",",
        encoding="UTF-8",
    )
    with open(csv_path, "rb") as f:
        job = client.load_table_from_file(f, table_id, job_config=job_config)
    job.result()
    table = client.get_table(table_id)
    print(f"Loaded {table_id}: {table.num_rows} rows")

# Ensure files exist
for fn in ["appointments_clean.csv","weather.csv","twitter_agg.csv","zip_city.csv"]:
    if not (DATA_DIR/fn).exists():
        raise FileNotFoundError(f"Missing {fn} in {DATA_DIR}. Upload or fix DATA_DIR.")

load_csv_to_bq("appointments_staging", str(DATA_DIR/"appointments_clean.csv"))
load_csv_to_bq("weather_staging", str(DATA_DIR/"weather.csv"))
load_csv_to_bq("twitter_staging", str(DATA_DIR/"twitter_agg.csv"))
load_csv_to_bq("zip_city_staging", str(DATA_DIR/"zip_city.csv"))


In [ ]:

ddl = '''
CREATE OR REPLACE TABLE `clinic_ops.appointments` AS
SELECT
  appt_id,
  DATE(date) AS dt,
  TIME(time) AS appt_time,
  UPPER(city) AS city,
  clinic,
  zip,
  visit_type,
  new_or_return,
  DATE(booked_date) AS booked_dt,
  status
FROM `clinic_ops.appointments_staging`;

CREATE OR REPLACE TABLE `clinic_ops.weather` AS
SELECT DATE(dt) AS dt, UPPER(city) AS city, tmax_c, tmin_c, prcp_mm
FROM `clinic_ops.weather_staging`;

CREATE OR REPLACE TABLE `clinic_ops.twitter` AS
SELECT DATE(dt) AS dt, UPPER(city) AS city, tweet_count_symptoms, sentiment_avg
FROM `clinic_ops.twitter_staging`;

CREATE OR REPLACE TABLE `clinic_ops.zip_city` AS
SELECT zip, UPPER(city) AS city
FROM `clinic_ops.zip_city_staging`;

CREATE OR REPLACE VIEW `clinic_ops.fact_day` AS
WITH base AS (
  SELECT
    a.dt, a.city, a.clinic, a.visit_type,
    IF(LOWER(a.new_or_return)='new', 1, 0) AS is_new_patient,
    IF(LOWER(a.status)='no_show', 1, 0) AS no_show,
    TIMESTAMP_DIFF(TIMESTAMP(a.dt, a.appt_time), TIMESTAMP(a.booked_dt, TIME '00:00:00'), DAY) AS lead_time_days,
    CASE
      WHEN EXTRACT(HOUR FROM a.appt_time) BETWEEN 7 AND 11 THEN 'AM'
      WHEN EXTRACT(HOUR FROM a.appt_time) BETWEEN 12 AND 16 THEN 'PM'
      ELSE 'Late'
    END AS hour_bucket
  FROM `clinic_ops.appointments` a
)
SELECT
  b.*,
  w.tmax_c, w.tmin_c, w.prcp_mm,
  CASE WHEN w.prcp_mm > 0 THEN 1 ELSE 0 END AS rain_flag,
  CASE WHEN w.tmax_c >= 30 THEN 'Hot' WHEN w.tmax_c <= 5 THEN 'Cold' ELSE 'Mild' END AS temp_bucket,
  t.tweet_count_symptoms, t.sentiment_avg
FROM base b
LEFT JOIN `clinic_ops.weather` w USING (dt, city)
LEFT JOIN `clinic_ops.twitter` t USING (dt, city);
'''
job = client.query(ddl)
job.result()
print("Standardized tables and view created.")


In [ ]:

import pandas as pd

# Lead time
q1 = '''
SELECT
  CASE WHEN lead_time_days <= 2 THEN '0-2'
       WHEN lead_time_days <= 7 THEN '3-7'
       ELSE '8+' END AS lead_bucket,
  ROUND(AVG(no_show)*100,1) AS no_show_rate_pct,
  COUNT(*) AS n
FROM `clinic_ops.fact_day`
GROUP BY lead_bucket
ORDER BY lead_bucket
'''
lead_bq = client.query(q1).to_dataframe()
display(lead_bq)

# Weather × time
q2 = '''
SELECT temp_bucket, hour_bucket,
       ROUND(AVG(no_show)*100,1) AS no_show_rate_pct,
       COUNT(*) AS n
FROM `clinic_ops.fact_day`
GROUP BY temp_bucket, hour_bucket
ORDER BY temp_bucket, hour_bucket
'''
wx_bq = client.query(q2).to_dataframe()
display(wx_bq)

# Chatter
q3 = '''
WITH with_bucket AS (
  SELECT *, CASE
    WHEN tweet_count_symptoms >= PERCENTILE_CONT(tweet_count_symptoms, 0.75) OVER() THEN 'High'
    ELSE 'Normal' END AS chatter_bucket
  FROM `clinic_ops.fact_day`
)
SELECT chatter_bucket,
       ROUND(AVG(no_show)*100,1) AS no_show_rate_pct,
       COUNT(*) AS n
FROM with_bucket
GROUP BY chatter_bucket
ORDER BY chatter_bucket
'''
chat_bq = client.query(q3).to_dataframe()
display(chat_bq)

# Top segments
q4 = '''
SELECT clinic, visit_type, hour_bucket, temp_bucket,
       ROUND(AVG(no_show)*100,1) AS no_show_rate_pct,
       COUNT(*) AS n
FROM `clinic_ops.fact_day`
GROUP BY clinic, visit_type, hour_bucket, temp_bucket
HAVING COUNT(*) >= 20
ORDER BY no_show_rate_pct DESC
LIMIT 15
'''
top_bq = client.query(q4).to_dataframe()
display(top_bq)

# Overall
q5 = 'SELECT ROUND(AVG(no_show)*100,1) AS overall_no_show_pct FROM `clinic_ops.fact_day`'
overall_bq = client.query(q5).to_dataframe()
display(overall_bq)


In [ ]:

# Save BigQuery outputs to the same OUTPUT_DIR for easy comparison
lead_bq.to_csv(OUTPUT_DIR/"leadtime_rates_bq.csv", index=False)
wx_bq.to_csv(OUTPUT_DIR/"weather_time_matrix_bq.csv", index=False)
chat_bq.to_csv(OUTPUT_DIR/"chatter_effect_bq.csv", index=False)
top_bq.to_csv(OUTPUT_DIR/"top_segments_bq.csv", index=False)
overall_bq.to_csv(OUTPUT_DIR/"overall_no_show_bq.csv", index=False)
print("Saved BigQuery outputs under:", OUTPUT_DIR)



### Optional: Load from **GCS** instead of local files
If your files are already in a GCS bucket, you can load via `gs://bucket/path.csv` URIs. Example:

```python
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)
uri = "gs://YOUR_BUCKET/clinic_project/appointments_clean.csv"
table_id = f"{PROJECT_ID}.clinic_ops.appointments_staging"
job = client.load_table_from_uri(uri, table_id, job_config=job_config)
job.result()
```


## (Colab) Download all outputs as a ZIP

In [ ]:

try:
    import shutil
    zip_path = "/content/clinic_verification_outputs.zip"
    shutil.make_archive("/content/clinic_verification_outputs", 'zip', OUTPUT_DIR)
    from google.colab import files
    files.download(zip_path)
except Exception:
    print("Zip/download skipped (not in Colab). Outputs are in:", OUTPUT_DIR)
